# पुनरावर्ती न्यूरल नेटवर्क

पिछले मॉड्यूल में, हमने टेक्स्ट के समृद्ध अर्थपूर्ण प्रतिनिधित्व को कवर किया। जिस आर्किटेक्चर का हम उपयोग कर रहे हैं, वह वाक्य में शब्दों के समग्र अर्थ को कैप्चर करता है, लेकिन यह शब्दों के **क्रम** को ध्यान में नहीं रखता है, क्योंकि एम्बेडिंग के बाद का समेकन ऑपरेशन मूल टेक्स्ट से इस जानकारी को हटा देता है। चूंकि ये मॉडल शब्दों के क्रम को प्रदर्शित करने में असमर्थ हैं, वे टेक्स्ट जनरेशन या प्रश्न उत्तर जैसे अधिक जटिल या अस्पष्ट कार्यों को हल नहीं कर सकते।

टेक्स्ट अनुक्रम के अर्थ को कैप्चर करने के लिए, हम एक न्यूरल नेटवर्क आर्किटेक्चर का उपयोग करेंगे जिसे **पुनरावर्ती न्यूरल नेटवर्क** या RNN कहा जाता है। RNN का उपयोग करते समय, हम अपने वाक्य को नेटवर्क के माध्यम से एक-एक टोकन करके पास करते हैं, और नेटवर्क कुछ **स्थिति** उत्पन्न करता है, जिसे हम अगले टोकन के साथ नेटवर्क में फिर से पास करते हैं।

![पुनरावर्ती न्यूरल नेटवर्क जनरेशन का एक उदाहरण दिखाने वाली छवि।](../../../../../lessons/5-NLP/16-RNN/images/rnn.png)

दिए गए टोकन अनुक्रम $X_0,\dots,X_n$ के लिए, RNN एक न्यूरल नेटवर्क ब्लॉकों का अनुक्रम बनाता है, और इस अनुक्रम को बैकप्रोपेगेशन का उपयोग करके एंड-टू-एंड ट्रेन करता है। प्रत्येक नेटवर्क ब्लॉक $(X_i,S_i)$ की एक जोड़ी को इनपुट के रूप में लेता है, और $S_{i+1}$ को परिणाम के रूप में उत्पन्न करता है। अंतिम स्थिति $S_n$ या आउटपुट $Y_n$ एक रैखिक वर्गीकरणकर्ता में जाती है ताकि परिणाम उत्पन्न हो सके। सभी नेटवर्क ब्लॉक समान वेट्स साझा करते हैं, और एक बैकप्रोपेगेशन पास का उपयोग करके एंड-टू-एंड ट्रेन किए जाते हैं।

> ऊपर दी गई आकृति पुनरावर्ती न्यूरल नेटवर्क को अनरोल्ड रूप (बाईं ओर) और अधिक कॉम्पैक्ट पुनरावर्ती प्रतिनिधित्व (दाईं ओर) में दिखाती है। यह समझना महत्वपूर्ण है कि सभी RNN सेल्स के समान **शेयर करने योग्य वेट्स** होते हैं।

चूंकि स्थिति वेक्टर $S_0,\dots,S_n$ नेटवर्क के माध्यम से पास किए जाते हैं, RNN शब्दों के बीच क्रमिक निर्भरता सीखने में सक्षम होता है। उदाहरण के लिए, जब अनुक्रम में कहीं *not* शब्द आता है, तो यह स्थिति वेक्टर के भीतर कुछ तत्वों को नकारना सीख सकता है।

अंदर, प्रत्येक RNN सेल में दो वेट मैट्रिक्स होते हैं: $W_H$ और $W_I$, और बायस $b$। प्रत्येक RNN चरण में, दिए गए इनपुट $X_i$ और इनपुट स्थिति $S_i$, आउटपुट स्थिति की गणना इस प्रकार की जाती है: $S_{i+1} = f(W_H\times S_i + W_I\times X_i+b)$, जहां $f$ एक सक्रियण फ़ंक्शन है (अक्सर $\tanh$)।

> टेक्स्ट जनरेशन (जिसे हम अगले यूनिट में कवर करेंगे) या मशीन अनुवाद जैसी समस्याओं के लिए, हम प्रत्येक RNN चरण में कुछ आउटपुट मान भी प्राप्त करना चाहते हैं। इस मामले में, एक और मैट्रिक्स $W_O$ होता है, और आउटपुट की गणना इस प्रकार की जाती है: $Y_i=f(W_O\times S_i+b_O)$।

आइए देखें कि पुनरावर्ती न्यूरल नेटवर्क हमारे समाचार डेटासेट को वर्गीकृत करने में कैसे मदद कर सकते हैं।

> सैंडबॉक्स वातावरण के लिए, हमें यह सुनिश्चित करने के लिए निम्नलिखित सेल चलाना होगा कि आवश्यक लाइब्रेरी इंस्टॉल हो गई है, और डेटा प्रीफेच हो गया है। यदि आप लोकल रूप से चला रहे हैं, तो आप निम्नलिखित सेल को छोड़ सकते हैं।


In [1]:
import sys
!{sys.executable} -m pip install --quiet tensorflow_datasets==4.4.0
!cd ~ && wget -q -O - https://mslearntensorflowlp.blob.core.windows.net/data/tfds-ag-news.tgz | tar xz

In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

# We are going to be training pretty large models. In order not to face errors, we need
# to set tensorflow option to grow GPU memory allocation when required
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

ds_train, ds_test = tfds.load('ag_news_subset').values()

जब बड़े मॉडल का प्रशिक्षण किया जाता है, तो GPU मेमोरी आवंटन एक समस्या बन सकता है। हमें विभिन्न मिनीबैच आकारों के साथ प्रयोग करने की आवश्यकता हो सकती है, ताकि डेटा हमारे GPU मेमोरी में फिट हो जाए और प्रशिक्षण पर्याप्त तेज़ हो। यदि आप इस कोड को अपने GPU मशीन पर चला रहे हैं, तो आप प्रशिक्षण को तेज़ करने के लिए मिनीबैच आकार को समायोजित करने के साथ प्रयोग कर सकते हैं।

> **Note**: NVidia ड्राइवरों के कुछ संस्करणों के बारे में ज्ञात है कि वे मॉडल का प्रशिक्षण समाप्त होने के बाद मेमोरी को रिलीज़ नहीं करते। हम इस नोटबुक में कई उदाहरण चला रहे हैं, और यह कुछ सेटअप में मेमोरी समाप्त होने का कारण बन सकता है, खासकर यदि आप इसी नोटबुक में अपने स्वयं के प्रयोग कर रहे हैं। यदि आप मॉडल का प्रशिक्षण शुरू करते समय कुछ अजीब त्रुटियों का सामना करते हैं, तो आप नोटबुक कर्नेल को पुनः आरंभ करना चाह सकते हैं।


In [3]:
batch_size = 16
embed_size = 64

## सरल RNN क्लासिफायर

सरल RNN के मामले में, प्रत्येक पुनरावर्ती यूनिट एक साधारण रैखिक नेटवर्क होता है, जो एक इनपुट वेक्टर और एक स्टेट वेक्टर लेता है, और एक नया स्टेट वेक्टर उत्पन्न करता है। Keras में, इसे `SimpleRNN` लेयर द्वारा दर्शाया जा सकता है।

हालांकि हम RNN लेयर को सीधे वन-हॉट एन्कोडेड टोकन पास कर सकते हैं, यह एक अच्छा विचार नहीं है क्योंकि उनकी उच्च आयामीयता होती है। इसलिए, हम शब्द वेक्टर की आयामीयता को कम करने के लिए एक एम्बेडिंग लेयर का उपयोग करेंगे, इसके बाद एक RNN लेयर और अंत में एक `Dense` क्लासिफायर।

> **Note**: उन मामलों में जहां आयामीयता इतनी अधिक नहीं होती, जैसे कि जब कैरेक्टर-लेवल टोकनाइज़ेशन का उपयोग किया जाता है, तो वन-हॉट एन्कोडेड टोकन को सीधे RNN सेल में पास करना समझदारी हो सकती है।


In [4]:
vocab_size = 20000

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=vocab_size,
    input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, embed_size),
    keras.layers.SimpleRNN(16),
    keras.layers.Dense(4,activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, None)              0         
_________________________________________________________________
embedding (Embedding)        (None, None, 64)          1280000   
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 16)                1296      
_________________________________________________________________
dense (Dense)                (None, 4)                 68        
Total params: 1,281,364
Trainable params: 1,281,364
Non-trainable params: 0
_________________________________________________________________


> **ध्यान दें:** यहाँ हम सरलता के लिए एक अप्रशिक्षित एम्बेडिंग लेयर का उपयोग कर रहे हैं, लेकिन बेहतर परिणामों के लिए हम Word2Vec का उपयोग करके एक पूर्व-प्रशिक्षित एम्बेडिंग लेयर का उपयोग कर सकते हैं, जैसा कि पिछले यूनिट में बताया गया है। यह आपके लिए एक अच्छा अभ्यास होगा कि आप इस कोड को पूर्व-प्रशिक्षित एम्बेडिंग के साथ काम करने के लिए अनुकूलित करें।

अब चलिए अपने RNN को प्रशिक्षित करते हैं। सामान्यतः RNN को प्रशिक्षित करना काफी कठिन होता है, क्योंकि जब RNN सेल्स को अनुक्रम की लंबाई के साथ अनरोल किया जाता है, तो बैकप्रोपेगेशन में शामिल लेयर्स की संख्या काफी अधिक हो जाती है। इसलिए हमें एक छोटा लर्निंग रेट चुनने की आवश्यकता होती है, और अच्छे परिणाम प्राप्त करने के लिए नेटवर्क को एक बड़े डेटासेट पर प्रशिक्षित करना होता है। इसमें काफी समय लग सकता है, इसलिए GPU का उपयोग करना बेहतर होता है।

प्रक्रिया को तेज़ करने के लिए, हम केवल समाचार शीर्षकों पर RNN मॉडल को प्रशिक्षित करेंगे और विवरण को छोड़ देंगे। आप विवरण के साथ प्रशिक्षण का प्रयास कर सकते हैं और देख सकते हैं कि क्या आप मॉडल को प्रशिक्षित कर सकते हैं।


In [5]:
def extract_title(x):
    return x['title']

def tupelize_title(x):
    return (extract_title(x),x['label'])

print('Training vectorizer')
vectorizer.adapt(ds_train.take(2000).map(extract_title))

Training vectorizer


In [6]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize_title).batch(batch_size),validation_data=ds_test.map(tupelize_title).batch(batch_size))

7500/7500 [==============================] - 82s 11ms/step - loss: 0.6629 - acc: 0.7623 - val_loss: 0.5559 - val_acc: 0.7995


> **नोट** कि सटीकता यहां कम हो सकती है, क्योंकि हम केवल समाचार शीर्षकों पर प्रशिक्षण कर रहे हैं।


## वेरिएबल अनुक्रमों पर पुनर्विचार 

याद रखें कि `TextVectorization` लेयर स्वचालित रूप से एक मिनीबैच में वेरिएबल लंबाई वाले अनुक्रमों को पैड टोकन के साथ पैड कर देती है। यह देखा गया है कि ये टोकन भी प्रशिक्षण में भाग लेते हैं, और वे मॉडल के कन्वर्जेंस को जटिल बना सकते हैं।

पैडिंग की मात्रा को कम करने के लिए हम कई तरीकों का उपयोग कर सकते हैं। उनमें से एक है डेटा सेट को अनुक्रम की लंबाई के अनुसार पुनः व्यवस्थित करना और सभी अनुक्रमों को उनके आकार के अनुसार समूहित करना। इसे `tf.data.experimental.bucket_by_sequence_length` फ़ंक्शन का उपयोग करके किया जा सकता है (देखें [डॉक्यूमेंटेशन](https://www.tensorflow.org/api_docs/python/tf/data/experimental/bucket_by_sequence_length))।

एक अन्य तरीका **मास्किंग** का उपयोग करना है। Keras में, कुछ लेयर अतिरिक्त इनपुट का समर्थन करती हैं जो दिखाती हैं कि प्रशिक्षण के दौरान किन टोकनों को ध्यान में रखा जाना चाहिए। मास्किंग को अपने मॉडल में शामिल करने के लिए, हम या तो एक अलग `Masking` लेयर ([डॉक्स](https://keras.io/api/layers/core_layers/masking/)) जोड़ सकते हैं, या हम अपने `Embedding` लेयर में `mask_zero=True` पैरामीटर निर्दिष्ट कर सकते हैं।

> **Note**: इस प्रशिक्षण में पूरे डेटा सेट पर एक एपोक पूरा करने में लगभग 5 मिनट लगेंगे। यदि आप धैर्य खो दें तो प्रशिक्षण को किसी भी समय रोकने के लिए स्वतंत्र महसूस करें। आप यह भी कर सकते हैं कि प्रशिक्षण के लिए उपयोग किए जाने वाले डेटा की मात्रा को सीमित करें, `ds_train` और `ds_test` डेटा सेट के बाद `.take(...)` क्लॉज जोड़कर।


In [7]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size,embed_size,mask_zero=True),
    keras.layers.SimpleRNN(16),
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

7500/7500 [==============================] - 371s 49ms/step - loss: 0.5401 - acc: 0.8079 - val_loss: 0.3780 - val_acc: 0.8822


अब जब हम मास्किंग का उपयोग कर रहे हैं, तो हम शीर्षकों और विवरणों के पूरे डेटासेट पर मॉडल को प्रशिक्षित कर सकते हैं।

> **नोट**: क्या आपने देखा है कि हम उस वेक्टराइज़र का उपयोग कर रहे हैं जिसे समाचार शीर्षकों पर प्रशिक्षित किया गया था, न कि लेख के पूरे मुख्य भाग पर? संभवतः, इससे कुछ टोकन अनदेखे हो सकते हैं, इसलिए वेक्टराइज़र को फिर से प्रशिक्षित करना बेहतर होगा। हालांकि, इसका प्रभाव बहुत छोटा हो सकता है, इसलिए हम सरलता के लिए पहले से प्रशिक्षित वेक्टराइज़र का उपयोग जारी रखेंगे।


## LSTM: लंबी अवधि की स्मृति

RNNs की मुख्य समस्याओं में से एक है **vanishing gradients**। RNNs काफी लंबे हो सकते हैं, और बैकप्रोपेगेशन के दौरान नेटवर्क की पहली परत तक ग्रेडिएंट्स को पूरी तरह से वापस ले जाना मुश्किल हो सकता है। जब ऐसा होता है, तो नेटवर्क दूरस्थ टोकन के बीच संबंधों को सीखने में असमर्थ हो जाता है। इस समस्या से बचने का एक तरीका है **स्पष्ट स्थिति प्रबंधन** को **गेट्स** का उपयोग करके लागू करना। गेट्स को पेश करने वाली दो सबसे सामान्य आर्किटेक्चर हैं **लंबी अवधि की स्मृति** (LSTM) और **गेटेड रिले यूनिट** (GRU)। यहां हम LSTMs को कवर करेंगे।

![लंबी अवधि की स्मृति सेल का एक उदाहरण दिखाने वाली छवि](../../../../../lessons/5-NLP/16-RNN/images/long-short-term-memory-cell.svg)

एक LSTM नेटवर्क को RNN के समान तरीके से व्यवस्थित किया जाता है, लेकिन इसमें दो अवस्थाएं होती हैं जो परत से परत तक पास की जाती हैं: वास्तविक स्थिति $c$, और छिपा हुआ वेक्टर $h$। प्रत्येक यूनिट पर, छिपा हुआ वेक्टर $h_{t-1}$ को इनपुट $x_t$ के साथ जोड़ा जाता है, और ये दोनों मिलकर **गेट्स** के माध्यम से स्थिति $c_t$ और आउटपुट $h_{t}$ पर नियंत्रण करते हैं। प्रत्येक गेट में सिग्मॉइड सक्रियता होती है (आउटपुट $[0,1]$ की सीमा में), जिसे स्थिति वेक्टर के साथ गुणा करने पर बिटवाइज मास्क के रूप में सोचा जा सकता है। LSTMs में निम्नलिखित गेट्स होते हैं (ऊपर दी गई तस्वीर में बाएं से दाएं):
* **भूल गेट** जो यह निर्धारित करता है कि वेक्टर $c_{t-1}$ के कौन से घटकों को हमें भूलना है, और कौन से पास करने हैं।
* **इनपुट गेट** जो यह तय करता है कि इनपुट वेक्टर और पिछले छिपे हुए वेक्टर से कितनी जानकारी को स्थिति वेक्टर में शामिल करना चाहिए।
* **आउटपुट गेट** जो नई स्थिति वेक्टर लेता है और तय करता है कि इसके कौन से घटकों का उपयोग नए छिपे हुए वेक्टर $h_t$ को उत्पन्न करने के लिए किया जाएगा।

स्थिति $c$ के घटकों को ऐसे फ्लैग्स के रूप में सोचा जा सकता है जिन्हें चालू और बंद किया जा सकता है। उदाहरण के लिए, जब हम अनुक्रम में नाम *Alice* का सामना करते हैं, तो हम अनुमान लगाते हैं कि यह एक महिला को संदर्भित करता है, और स्थिति में वह फ्लैग उठाते हैं जो कहता है कि हमारे पास वाक्य में एक स्त्रीलिंग संज्ञा है। जब हम आगे *and Tom* शब्दों का सामना करते हैं, तो हम वह फ्लैग उठाते हैं जो कहता है कि हमारे पास बहुवचन संज्ञा है। इस प्रकार स्थिति में हेरफेर करके हम वाक्य के व्याकरणिक गुणों का ट्रैक रख सकते हैं।

> **Note**: LSTMs की आंतरिक संरचना को समझने के लिए यहां एक शानदार संसाधन है: [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) क्रिस्टोफर ओलाह द्वारा।

हालांकि LSTM सेल की आंतरिक संरचना जटिल लग सकती है, Keras इस कार्यान्वयन को `LSTM` लेयर के अंदर छुपा देता है, इसलिए ऊपर दिए गए उदाहरण में हमें केवल पुनरावर्ती लेयर को बदलने की आवश्यकता है:


In [8]:
model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, embed_size),
    keras.layers.LSTM(8),
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(8),validation_data=ds_test.map(tupelize).batch(8))

15000/15000 [==============================] - 188s 13ms/step - loss: 0.5692 - acc: 0.7916 - val_loss: 0.3441 - val_acc: 0.8870


## द्विदिश और बहुस्तरीय RNNs

अब तक के हमारे उदाहरणों में, पुनरावर्ती नेटवर्क एक अनुक्रम की शुरुआत से अंत तक काम करते हैं। यह हमें स्वाभाविक लगता है क्योंकि यह उसी दिशा का अनुसरण करता है जिसमें हम पढ़ते हैं या भाषण सुनते हैं। हालांकि, उन परिस्थितियों के लिए जहां इनपुट अनुक्रम का रैंडम एक्सेस आवश्यक है, दोनों दिशाओं में पुनरावर्ती गणना चलाना अधिक समझदारी भरा होता है। ऐसे RNNs जो दोनों दिशाओं में गणना की अनुमति देते हैं, उन्हें **द्विदिश** RNNs कहा जाता है, और इन्हें एक विशेष `Bidirectional` लेयर के साथ पुनरावर्ती लेयर को लपेटकर बनाया जा सकता है।

> **Note**: `Bidirectional` लेयर अपनी भीतर की लेयर की दो प्रतियां बनाती है और उनमें से एक की `go_backwards` प्रॉपर्टी को `True` सेट करती है, जिससे वह अनुक्रम के साथ विपरीत दिशा में जाती है।

पुनरावर्ती नेटवर्क, चाहे एक दिशा में हो या द्विदिश, अनुक्रम के भीतर पैटर्न को कैप्चर करते हैं और उन्हें स्टेट वेक्टर में स्टोर करते हैं या आउटपुट के रूप में लौटाते हैं। जैसे कि कन्वोल्यूशनल नेटवर्क्स में होता है, हम पहले लेयर द्वारा निकाले गए निम्न स्तर के पैटर्न से उच्च स्तर के पैटर्न कैप्चर करने के लिए पहले लेयर के बाद एक और पुनरावर्ती लेयर बना सकते हैं। यह हमें **बहुस्तरीय RNN** की अवधारणा तक ले जाता है, जिसमें दो या अधिक पुनरावर्ती नेटवर्क होते हैं, जहां पिछले लेयर का आउटपुट अगले लेयर को इनपुट के रूप में दिया जाता है।

![मल्टीलेयर लॉन्ग-शॉर्ट-टर्म-मेमोरी RNN दिखाने वाली छवि](../../../../../lessons/5-NLP/16-RNN/images/multi-layer-lstm.jpg)

*फर्नांडो लोपेज़ द्वारा [इस शानदार पोस्ट](https://towardsdatascience.com/from-a-lstm-cell-to-a-multilayer-lstm-network-with-pytorch-2899eb5696f3) से ली गई तस्वीर।*

Keras इन नेटवर्क्स को बनाना आसान बनाता है, क्योंकि आपको बस मॉडल में अधिक पुनरावर्ती लेयर जोड़नी होती है। अंतिम लेयर को छोड़कर सभी लेयर के लिए, हमें `return_sequences=True` पैरामीटर निर्दिष्ट करना होता है, क्योंकि हमें लेयर से सभी मध्यवर्ती स्टेट्स चाहिए, न कि केवल पुनरावर्ती गणना की अंतिम स्टेट।

आइए हमारे वर्गीकरण समस्या के लिए एक दो-लेयर द्विदिश LSTM बनाते हैं।

> **Note** यह कोड फिर से पूरा होने में काफी समय लेता है, लेकिन यह हमें अब तक देखी गई सबसे अधिक सटीकता देता है। तो शायद इंतजार करना और परिणाम देखना उचित हो सकता है।


In [9]:
model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, 128, mask_zero=True),
    keras.layers.Bidirectional(keras.layers.LSTM(64,return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),    
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

5045/7500 [===================>..........] - ETA: 2:33 - loss: 0.3709 - acc: 0.8706

## अन्य कार्यों के लिए RNNs

अब तक, हमने RNNs का उपयोग टेक्स्ट अनुक्रमों को वर्गीकृत करने के लिए किया है। लेकिन वे और भी कई कार्य संभाल सकते हैं, जैसे टेक्स्ट जनरेशन और मशीन ट्रांसलेशन — हम इन कार्यों पर अगले यूनिट में विचार करेंगे।



---

**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। जबकि हम सटीकता सुनिश्चित करने का प्रयास करते हैं, कृपया ध्यान दें कि स्वचालित अनुवाद में त्रुटियां या अशुद्धियां हो सकती हैं। मूल भाषा में उपलब्ध मूल दस्तावेज़ को प्रामाणिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सिफारिश की जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम जिम्मेदार नहीं हैं।
